In [1]:
from core import *
from keras.models import Model
from keras.layers import Conv2D, Input, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
import os
import glob

Using TensorFlow backend.


In [2]:
DATA_PATH = "../Data/Median_images"

In [3]:
def model(input_shape):
    
    X_input = Input(input_shape)
    
    X = Conv2D(filters=128,kernel_size=(9,9), kernel_initializer ='glorot_uniform', 
                     activation='relu', padding ='same', bias=True, input_shape=input_shape)(X_input)
    
    X = Conv2D(filters = 64, kernel_size=(3,3), kernel_initializer='glorot_uniform',
                     activation='relu', padding='same', bias=True)(X)

    X = Conv2D(filters=1, kernel_size=(5,5), kernel_initializer='glorot_uniform',
                     activation='linear', padding='same', bias=True)(X)
    
    
    model = Model(inputs=X_input, outputs=X, name='SRCNN model')

    return model


def psnr(ref,target):
    target_data = np.array(target, dtype=float)
    ref_data = np.array(ref, dtype=float)

    diff = ref_data - target_data
    diff = diff.flatten('C')

    rmse = math.sqrt(np.mean(diff ** 2.))

    return 20 * math.log10(1. / rmse)

In [4]:
Y_imgs_path = [path for path in glob.glob(DATA_PATH + "/*_HR.png")]
X_imgs_path = [path[:-7] + ".png" for path in Y_imgs_path]

In [5]:
Y_imgs_path[0].split('/')[-1][:-7]

'imgset0213'

In [6]:
X_imgs = np.array([skimage.img_as_float64(load_image(path, dtype=np.uint16)) for path in X_imgs_path])
Y_imgs = np.array([skimage.img_as_float64(load_image(path, dtype=np.uint16)) for path in Y_imgs_path])

In [7]:
X_imgs = X_imgs.reshape(X_imgs.shape[0],X_imgs.shape[1],X_imgs.shape[2],1)
Y_imgs = Y_imgs.reshape(Y_imgs.shape[0],Y_imgs.shape[1],Y_imgs.shape[2],1)


In [8]:
srcnn_model = model(X_imgs.shape[1:])
srcnn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error']) 
srcnn_model.load_weights("../models/SRCNN/weights/weights.h5")   


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=128, kernel_size=(9, 9), kernel_initializer="glorot_uniform", activation="relu", padding="same", input_shape=(384, 384,..., use_bias=True)`
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=64, kernel_size=(3, 3), kernel_initializer="glorot_uniform", activation="relu", padding="same", use_bias=True)`
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(filters=1, kernel_size=(5, 5), kernel_initializer="glorot_uniform", activation="linear", padding="same", use_bias=True)`
  if sys.path[0] == '':


In [9]:
srcnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 384, 384, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 384, 384, 128)     10496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 384, 384, 64)      73792     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 384, 384, 1)       1601      
Total params: 85,889
Trainable params: 85,889
Non-trainable params: 0
_________________________________________________________________


In [10]:
# srcnn_model.fit(X_imgs, Y_imgs, batch_size=32, shuffle=True, nb_epoch=200, verbose=1)

In [11]:
Z_imgs_path = [path[:-6] + "SM.png" for path in Y_imgs_path]
# Z_imgs = np.array([load_image(path, dtype=np.bool) for path in Z_imgs_path])
# for x_img,img_id in zip(X_imgs,Y_imgs_path):
#     img_id = img_id.split('/')[-1][:-7]
#     y_img_path = "../Data/train/"

In [12]:
score = []
count = 0
for x,y,z_path in zip(X_imgs,Y_imgs,Z_imgs_path):
    x = x.reshape(1,x.shape[0],x.shape[1],x.shape[2])
    z = load_image(z_path,dtype = np.bool)
    scene_id = z_path.split('/')[-1][:-7]
    pred = srcnn_model.predict(x)[0]
    pred = np.float64(pred)
    score.append(test_score_image(sr = pred,hr = y,sm = z,scene_id = scene_id))
    count += 1
    print(count)
print(np.mean(score))
    

ValueError: 